# Seattle Capstone Project


Load libraries

In [1]:
!pip install -U numpy

!pip install -U pandas

!pip install -U scipy

!pip install -U scikit-learn

!pip install -U imbalanced-learn

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Libraries imported.')



Solving environment: done

# All requested packages already installed.

Libraries imported.


### Download Dataset

In [3]:
#neighborhoods = pd.read_csv("seattle_district_trial.csv")
#neighborhoods.head()

In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [5]:
#Download datasets (from wikipedia) and also the geo coordinates csv file. 
url= 'https://docs.google.com/spreadsheets/d/1vO_mPBbxWIt0cnhicr_SkKgSLA_I1qdCKvQyNV87A-c/edit#gid=0'
neighborhoods = pd.read_html(url, header=1)[0]

#geo = pd.read_csv("Geospatial_Coordinates.csv")

neighborhoods.head()
del neighborhoods['1']
neighborhoods.head()

,seattle_district,latitude,longitude
0,Atlantic,47.5972,-122.309
1,Ballard,47.6792,-122.386
2,Beacon Hill,47.5526,-122.301
3,Capitol Hill,47.6253,-122.322
4,Central Seattle,47.6088,-122.296


In [6]:
#get the geo coordinates of Seattle, WA
address = 'Seattle, WA'

geolocator = Nominatim(user_agent="wa_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Seattle are {}, {}.'.format(latitude, longitude))
#pause here 2:!5

The geograpical coordinates of Seattle are 47.6038321, -122.3300624.


In [7]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install folium # map rendering library
#!pip install beautifulsoup4 


!pip install bs4
!pip install lxml
!pip install html5lib
import folium 

In [8]:
# create map of Seattle using latitude and longitude values
map_seattle = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(neighborhoods['latitude'], neighborhoods['longitude'], neighborhoods['seattle_district']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_seattle)  
    
map_seattle

In [9]:
# @hidden_cell
CLIENT_ID = 'R2XCHP4ZWLG1E4EGN4R24V0PME3HGXGUPU3JEF2JW1HXDCAM' # your Foursquare ID
CLIENT_SECRET = 'YGBNXFLJFKA2HBKM2X1FUXPOBUIFXXXAVVU4VRXUM1PTU5QI' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

#print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

In [10]:
atlantic_data = neighborhoods[neighborhoods['seattle_district'] == 'Atlantic'].reset_index(drop=True)
atlantic_data.head()

neighborhood_latitude = atlantic_data.loc[0, 'latitude'] # neighborhood latitude value
neighborhood_longitude = atlantic_data.loc[0, 'longitude'] # neighborhood longitude value

neighborhood_name = atlantic_data.loc[0, 'seattle_district'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Atlantic are 47.5972, -122.309.


In [11]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 #define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=R2XCHP4ZWLG1E4EGN4R24V0PME3HGXGUPU3JEF2JW1HXDCAM&client_secret=YGBNXFLJFKA2HBKM2X1FUXPOBUIFXXXAVVU4VRXUM1PTU5QI&v=20180605&ll=47.5972,-122.309&radius=500&limit=100'

In [12]:
results = requests.get(url).json()
#results

In [13]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [14]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns] #throws an error

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Humble Pie,Pizza Place,47.597635,-122.313205
1,Cheeky Cafe,Breakfast Spot,47.599404,-122.310048
2,Seattle Bouldering Project,Climbing Gym,47.593505,-122.310925
3,Starbucks,Coffee Shop,47.597098,-122.312493
4,Pratt Fine Arts Center,Art Gallery,47.599731,-122.307209


In [15]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

37 venues were returned by Foursquare.


### Explore Districts in Seattle

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
neighborhoods_venues = getNearbyVenues(names=neighborhoods['seattle_district'],
                                   latitudes=neighborhoods['latitude'],
                                   longitudes=neighborhoods['longitude']
                                  )


Atlantic
Ballard
Beacon Hill
Capitol Hill
Central Seattle
Columbia City
Delridge
Downtown
Harrison/Denny
Industrial District
Madison Park
Madrona
Magnolia
Northgate
Queen Anne
Rainier Valley
Seward Park
South Lake Union
University District
West Seattle
Windermere
Belltown
Fremont
Leschi
Mount Baker


In [18]:
print(neighborhoods_venues.shape)
neighborhoods_venues.head()

(807, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Atlantic,47.5972,-122.309,Humble Pie,47.597635,-122.313205,Pizza Place
1,Atlantic,47.5972,-122.309,Cheeky Cafe,47.599404,-122.310048,Breakfast Spot
2,Atlantic,47.5972,-122.309,Seattle Bouldering Project,47.593505,-122.310925,Climbing Gym
3,Atlantic,47.5972,-122.309,Starbucks,47.597098,-122.312493,Coffee Shop
4,Atlantic,47.5972,-122.309,Pratt Fine Arts Center,47.599731,-122.307209,Art Gallery


In [19]:
neighborhoods_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Atlantic,37,37,37,37,37,37
Ballard,4,4,4,4,4,4
Beacon Hill,2,2,2,2,2,2
Belltown,100,100,100,100,100,100
Capitol Hill,37,37,37,37,37,37
Central Seattle,18,18,18,18,18,18
Columbia City,39,39,39,39,39,39
Delridge,4,4,4,4,4,4
Downtown,100,100,100,100,100,100


In [20]:
print('There are {} uniques categories.'.format(len(neighborhoods_venues['Venue Category'].unique())))

There are 215 uniques categories.


### Analyze each neighborhood

In [21]:
# one hot encoding
seattle_onehot = pd.get_dummies(neighborhoods_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
seattle_onehot['Neighborhood'] = neighborhoods_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [seattle_onehot.columns[-1]] + list(seattle_onehot.columns[:-1])
seattle_onehot = seattle_onehot[fixed_columns]

seattle_onehot.head()

,Yoga Studio,ATM,Accessories Store,African Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Dealership,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Store,Big Box Store,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Bus Line,Bus Station,Bus Stop,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Chiropractor,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Ethiopian Restaurant,Fair,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Historic Site,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Marijuana Dispensary,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Motel,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Perfume Shop,Pet Service,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Service,Residential Building (Apartment / Condo),Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Salsa Club,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Smoothie Shop,Snack Place,Soccer Field,Social Club,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Tour Provider,Toy / Game Store,Track,Trail,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Water Park,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Atlantic,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Atlantic,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [22]:
#Examine the new dataframe size.

seattle_onehot.shape

(807, 215)

In [23]:
#Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
seattle_grouped = seattle_onehot.groupby('Neighborhood').mean().reset_index()
seattle_grouped

,Neighborhood,Yoga Studio,ATM,Accessories Store,African Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Dealership,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Store,Big Box Store,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Bus Line,Bus Station,Bus Stop,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Chiropractor,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Ethiopian Restaurant,Fair,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Historic Site,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Marijuana Dispensary,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Motel,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Perfume Shop,Pet Service,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Service,Residential Building (Apartment / Condo),Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Salsa Club,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Smoothie Shop,Snack Place,Soccer Field,Social Club,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Tour Provider,Toy / Game Store,Track,Trail,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Water Park,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store
0,Atlantic,0.027027,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027027,0.00,0.000000,0.027027,0.00,0.000000,0.054054,0.000000,0.000000,0.00000,0.0000,0.000000,0.027027,0.027027,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.027027,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.027027,0.027027,0.000000,0.000000,0.027027,0.000000,0.000000,0.027027,0.000000,0.000000,0.054054,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00,0.00,0.000000,0.000000,0.027027,0.000000,0.027027,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.027027,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027027,0.000000,0.000000,0.0000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027027,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00

In [24]:
#size of dataset
seattle_grouped.shape

(25, 215)

### What are the top 5 commonn venues in each neighborhood?

In [25]:
num_top_venues = 5

for hood in seattle_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = seattle_grouped[seattle_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Atlantic----
                   venue  freq
0  Vietnamese Restaurant  0.14
1                 Bakery  0.05
2            Pizza Place  0.05
3                  Plaza  0.05
4            Coffee Shop  0.05


----Ballard----
               venue  freq
0      Jewelry Store  0.25
1  French Restaurant  0.25
2               Park  0.25
3        Coffee Shop  0.25
4        Yoga Studio  0.00


----Beacon Hill----
                venue  freq
0  Light Rail Station   0.5
1         Bus Station   0.5
2         Yoga Studio   0.0
3          Playground   0.0
4   Outdoor Sculpture   0.0


----Belltown----
              venue  freq
0               Bar  0.06
1       Pizza Place  0.05
2  Sushi Restaurant  0.05
3       Coffee Shop  0.04
4    Breakfast Spot  0.04


----Capitol Hill----
                 venue  freq
0         Cocktail Bar  0.11
1      Thai Restaurant  0.08
2                  Bar  0.08
3  American Restaurant  0.08
4          Coffee Shop  0.08


----Central Seattle----
                  venue  freq

In [26]:
# Stick that into a pandas dataframe
## First, let's write a function to sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
#Make new datafram and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = seattle_grouped['Neighborhood']

for ind in np.arange(seattle_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(seattle_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Atlantic,Vietnamese Restaurant,Bakery,Pizza Place,Plaza,Coffee Shop,Residential Building (Apartment / Condo),Thrift / Vintage Store,Yoga Studio,Bus Stop,Café
1,Ballard,Jewelry Store,French Restaurant,Park,Coffee Shop,Yoga Studio,Pizza Place,Outdoor Sculpture,Paper / Office Supplies Store,Pastry Shop,Performing Arts Venue
2,Beacon Hill,Light Rail Station,Bus Station,Yoga Studio,Playground,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Perfume Shop
3,Belltown,Bar,Pizza Place,Sushi Restaurant,Coffee Shop,Breakfast Spot,Cocktail Bar,Gym,Italian Restaurant,Seafood Restaurant,Bakery
4,Capitol Hill,Cocktail Bar,Thai Restaurant,Bar,American Restaurant,Coffee Shop,Italian Restaurant,Yoga Studio,Szechuan Restaurant,Noodle House,Gym


In [28]:
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Atlantic,Vietnamese Restaurant,Bakery,Pizza Place,Plaza,Coffee Shop,Residential Building (Apartment / Condo),Thrift / Vintage Store,Yoga Studio,Bus Stop,Café
1,Ballard,Jewelry Store,French Restaurant,Park,Coffee Shop,Yoga Studio,Pizza Place,Outdoor Sculpture,Paper / Office Supplies Store,Pastry Shop,Performing Arts Venue
2,Beacon Hill,Light Rail Station,Bus Station,Yoga Studio,Playground,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Perfume Shop
3,Belltown,Bar,Pizza Place,Sushi Restaurant,Coffee Shop,Breakfast Spot,Cocktail Bar,Gym,Italian Restaurant,Seafood Restaurant,Bakery
4,Capitol Hill,Cocktail Bar,Thai Restaurant,Bar,American Restaurant,Coffee Shop,Italian Restaurant,Yoga Studio,Szechuan Restaurant,Noodle House,Gym
5,Central Seattle,Ethiopian Restaurant,Deli / Bodega,Farmers Market,Bakery,Coworking Space,Playground,Fried Chicken Joint,Bus Stop,Tennis Court,Thai Restaurant
6,Columbia City,Coffee Shop,Pizza Place,Mexican Restaurant,Diner,Ethiopian Restaurant,Farmers Market,Fried Chicken Joint,Gas Station,Gastropub,Gym
7,Delridge,Convenience Store,Theater,Café,Yoga Studio,Pharmacy,Organic Grocery,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop
8,Downtown,Hotel,Coffee Shop,Cocktail Bar,Seafood Restaurant,Sushi Restaurant,American Restaurant,Sandwich Place,Clothing Store,Japanese Restaurant,Bar
9,Fremont,Coffee Shop,Bar,Thai Restaurant,Dessert Shop,Pub,Music Venue,Outdoor Sculpture,Park,Pizza Place,Boutique


In [29]:
num_venues = neighborhoods_venues.groupby('Neighborhood').count()

#Drop neighborhoods with less than 10 venues

df_filtered = num_venues[num_venues['Venue'] >= 10]

In [30]:
df_filtered

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Atlantic,37,37,37,37,37,37
Belltown,100,100,100,100,100,100
Capitol Hill,37,37,37,37,37,37
Central Seattle,18,18,18,18,18,18
Columbia City,39,39,39,39,39,39
Downtown,100,100,100,100,100,100
Fremont,83,83,83,83,83,83
Industrial District,17,17,17,17,17,17
Leschi,10,10,10,10,10,10


In [31]:
!pip install sklearn

  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1316 sha256=00d9145b6acddca45b14dd5fcbb752bfd478704bfd6354a17c65a4a1a02f3cf2
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn


In [32]:
from sklearn.cluster import KMeans

# Cluster Neighborhoods
Run K-means to cluster neighborhood into 5 clusters

In [33]:
# set number of clusters
kclusters = 5

seattle_grouped_clustering = seattle_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(seattle_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


array([1, 1, 2, 1, 1, 1, 1, 3, 1, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [34]:
#Clustering labels can only be run once or else will throw an error "already made" basically

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)




In [35]:
seattle_merged = neighborhood

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
seattle_merged = seattle_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

seattle_merged.head() # check the last columns!

TypeError: join() takes no keyword arguments